In [1]:
import gspread
import pymongo as pm
import pandas as pd
import numpy as np
from gspread_dataframe import set_with_dataframe
from datetime import datetime, timedelta

# Create the client
db_client = pm.MongoClient(
    '####')
db = db_client.mygig

gc = gspread.service_account(
    '####'')
T1 = gc.open_by_key('####')
df = pd.read_csv('####')

In [2]:
users = pd.DataFrame(db['users'].find(
    {'phone': {'$in': list(df['№ тел'].astype('float64'))}}))
workers = pd.DataFrame(db['workers'].find(
    {'user': {'$in': list(users['_id'])}}))
jobs = pd.DataFrame(db['jobs'].find({'worker': {'$in': list(workers['_id'])}}))
job_history = pd.DataFrame(db['job_history'].find(
    {'job': {'$in': list(jobs['_id'])}}))
workplaces = pd.DataFrame(db['workplaces'].find(
    {'_id': {'$in': list(jobs['workplace'])}}))
contractors = pd.DataFrame(db['contractors'].find(
    {'_id': {'$in': list(workplaces['contractor'])}}))

In [3]:
users['phone'] = users['phone'].fillna(0).astype('int64').astype(str)

In [4]:
# Присоединяем по номеру телефона все данные
df = df.merge(users[['_id', 'phone']].rename(columns={'_id': 'user_id'}),
              left_on=df['№ тел'].astype(str), right_on='phone', how='left').merge(
    workers[['_id', 'user']].rename(columns={'_id': 'worker_id'}),
    left_on='user_id', right_on='user', how='left').merge(
    jobs[jobs['status'] != 'cancelled'][['_id', 'worker', 'status', 'start',
                                         'workplace', 'api_client', 'client']].rename(columns={
                                             '_id': 'job_id'}),
    left_on='worker_id', right_on='worker', how='left').merge(
    job_history[job_history['status'] == 'booked'][['job', 'createdAt']],
    left_on='job_id', right_on='job', how='left').merge(
    workplaces[['_id', 'contractor']],
    left_on='workplace', right_on='_id', how='left').merge(
    contractors[['_id', 'type', 'legal_name']],
    left_on='contractor', right_on='_id', how='left')[[
        'ФИО', '№ тел', 'job_id', 'status', 'start', 'createdAt',
        'type', 'api_client', 'legal_name', 'client']]

# Формируем итоговый датафрейм
df.columns = ['ФИО', '№ тел', 'id смены', 'статус смены', 'дата смены', 'дата бронирования', 'тип',
              'способ бронирвоания смены', 'Контрагент', 'Клиент']

# Загружаем данные на лист
T1.get_worksheet(1).clear()
set_with_dataframe(T1.get_worksheet(1), df)